<a href="https://colab.research.google.com/github/Guillermo-rv/ML/blob/main/mle_rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
!pip install import-ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
# Step 1: Loading the data
df_hm = pd.read_csv("drive/MyDrive/Colab_Notebooks/data/hm2018.csv")
df_articles = pd.read_csv("drive/MyDrive/Colab_Notebooks/data/articles.csv")
print("Unique values for object type articles:")
#print(df_articles.astype('object').describe().loc['unique',:])
print(df_articles.nunique())

In [ ]:
print(df_articles.nunique())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


article_id                      105542
product_code                     47224
prod_name                        45875
product_type_no                    132
product_type_name                  131
product_group_name                  19
graphical_appearance_no             30
graphical_appearance_name           30
colour_group_code                   50
colour_group_name                   50
perceived_colour_value_id            8
perceived_colour_value_name          8
perceived_colour_master_id          20
perceived_colour_master_name        20
department_no                      299
department_name                    250
index_code                          10
index_name                          10
index_group_no                       5
index_group_name                     5
section_no                          57
section_name                        56
garment_group_no                    21
garment_group_name                  21
detail_desc                      43404
dtype: int64


In [ ]:
len(df_articles)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


105542

In [ ]:
df_articles.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [ ]:
df_hm.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [ ]:
def preprocessing_hm(df_transactions, df_articles, column_select, num_unique_articles=1):
  # Merge the 2 dataframes by 'article_id' and selecting only the 'column_select' column
  df_merge = pd.merge(df_transactions, df_articles[['article_id',column_select]], on='article_id', how='inner')

  # Merge by customer and day and keep the information from column_select in list type
  df_merge = df_merge.groupby(by=['customer_id','t_dat'])[column_select].agg(list)
  df_merge = pd.DataFrame(df_merge).reset_index()

  # Select only the rows with more than "num_unique_articles" articles bought in the same day
  df_merge['len_unique'] = df_merge[column_select].apply(set).apply(len)
  df_merge = df_merge[df_merge['len_unique']>num_unique_articles]
  df_merge = df_merge.drop(columns=['len_unique'])
  df_merge = df_merge.reset_index(drop=True)

  return df_merge

def preprocessing_hm_transactionEncoder(df_transactions, df_articles, column_select, num_unique_articles=1):
  # Running the previous function
  df_preprocessing = preprocessing_hm(df_transactions, df_articles, column_select, num_unique_articles)

  # Transforming the df_preprocessing using a TransactionEncoder
  t_encoder = TransactionEncoder()
  df_encoder = t_encoder.fit_transform(df_preprocessing[column_select])
  df_encoder = pd.DataFrame(df_encoder, columns=t_encoder.columns_)
  return df_encoder

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from mlxtend.preprocessing import TransactionEncoder

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Measure time and memory
import time
import tracemalloc # memory
start_time = time.time()
tracemalloc.start()

df_hm2018 = preprocessing_hm_transactionEncoder(df_hm, df_articles, 'product_type_name', 10)

end_time = time.time()
print("Time spent during the execution: " + str(end_time - start_time))
print("Memory spent during the execution: " + str(tracemalloc.get_traced_memory())) # (current, peak)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Time spent during the execution: 52.88195443153381
Memory spent during the execution: (221866, 389486120)


##  2

In [ ]:
def applying_assoc_rules(df_preprocessed, my_min_support, my_metric, my_threshold_value):
  # Apriori algorithm --> frequent itemsets --> min_support
  frequentItemsets = apriori(df_preprocessed, min_support = my_min_support, use_colnames = True)

  # Association rules --> my_metric & my_threshold_value (confidence, lift, leverage, ...)
  rules = association_rules(frequentItemsets, metric = my_metric, min_threshold = my_threshold_value)

  return frequentItemsets, rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Running the previous function
freq2018, rules2018 = applying_assoc_rules(df_hm2018, 0.4, 'confidence', 0.4)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rules2018

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Blouse),(Sweater),0.543871,0.801935,0.440645,0.810202,1.010308,0.004496,1.043552,0.022368
1,(Sweater),(Blouse),0.801935,0.543871,0.440645,0.549477,1.010308,0.004496,1.012444,0.051512
2,(Blouse),(Trousers),0.543871,0.840000,0.464516,0.854093,1.016777,0.007665,1.096585,0.036174
3,(Trousers),(Blouse),0.840000,0.543871,0.464516,0.552995,1.016777,0.007665,1.020412,0.103125
4,(Dress),(Sweater),0.665161,0.801935,0.516129,0.775946,0.967591,-0.017287,0.884002,-0.090935
5,(Sweater),(Dress),0.801935,0.665161,0.516129,0.643604,0.967591,-0.017287,0.939514,-0.144647
6,(Dress),(T-shirt),0.665161,0.649032,0.416774,0.626576,0.965401,-0.014937,0.939864,-0.096686
7,(T-shirt),(Dress),0.649032,0.665161,0.416774,0.642147,0.965401,-0.014937,0.935688,-0.092654
8,(Top),(Dress),0.596129,0.665161,0.409677,0.687229,1.033177,0.013155,1.070557,0.079510
9,(Dress),(Top),0.665161,0.596129,0.409677,0.615907,1.033177,0.013155,1.051492,0.095902


In [ ]:
rules2018[rules2018['lift']>1.5].sort_values(ascending=False, by='lift')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
226,(Bikini top),"(Dress, Swimwear bottom)",0.199355,0.146452,0.121290,0.608414,4.154370,0.092094,2.179723,0.948347
224,"(Dress, Swimwear bottom)",(Bikini top),0.146452,0.199355,0.121290,0.828194,4.154370,0.092094,4.660165,0.889568
243,(Swimwear bottom),"(Bikini top, T-shirt)",0.214194,0.127742,0.112258,0.524096,4.102775,0.084897,1.832846,0.962403
239,"(Bikini top, T-shirt)",(Swimwear bottom),0.127742,0.214194,0.112258,0.878788,4.102775,0.084897,6.482903,0.867017
245,"(Top, Swimwear bottom)",(Bikini top),0.133548,0.199355,0.106452,0.797101,3.998405,0.079828,3.946037,0.865484
247,(Bikini top),"(Top, Swimwear bottom)",0.199355,0.133548,0.106452,0.533981,3.998405,0.079828,1.859261,0.936620
248,(Swimwear bottom),"(Top, Bikini top)",0.214194,0.124516,0.106452,0.496988,3.991354,0.079781,1.740483,0.953744
244,"(Top, Bikini top)",(Swimwear bottom),0.124516,0.214194,0.106452,0.854922,3.991354,0.079781,5.416452,0.856051
9,(Bikini top),(Swimwear bottom),0.199355,0.214194,0.170323,0.854369,3.988771,0.127622,5.395871,0.935866
10,(Swimwear bottom),(Bikini top),0.214194,0.199355,0.170323,0.795181,3.988771,0.127622,3.909032,0.953538
